In [3]:
from lxml import etree

In [4]:
try:
    from lxml  import etree
except ImportError:
    import xml.etree.ElementTree as etree
    print("running with Python's xml.etree.ElementTree")

Element 类

In [5]:
root = etree.Element("root")
root

<Element root at 0x17ad8f4de80>

元素的 XML 标签名称通过 tag 属性访问

In [6]:
print(root.tag)

root


元素以 XML 树结构进行组织。要创建子元素并将它们添加到父元素，可以使用append()方法

In [7]:
root.append(etree.Element("child1"))

然而，这种情况非常常见，因此有一种更短且更有效的方法可以做到这一点：SubElement 工厂。它接受与 Element 工厂相同的参数，但还需要父级作为第一个参数

In [8]:
child2 = etree.SubElement(root, "child2")
child3 = etree.SubElement(root, "child3")

要查看这确实是 XML，您可以序列化您创建的树

In [9]:
etree.tostring(root)

b'<root><child1/><child2/><child3/></root>'

In [10]:
def prettyprint(element, **kwargs):
    xml = etree.tostring(element, pretty_print=True, **kwargs)
    print(xml.decode(), end='')

In [11]:
prettyprint(root)

<root>
  <child1/>
  <child2/>
  <child3/>
</root>


元素是列表
为了使对这些子元素的访问变得简单直接，元素尽可能模仿普通 Python 列表的行为

In [12]:
child = root[0]
print(child.tag)

child1


In [13]:
print(len(root))

3


In [14]:
root.index(root[1])

1

In [15]:
children = list(root)

In [16]:
for child in children:
    print(child.tag)

child1
child2
child3


In [17]:
root.insert(0, etree.Element("child0"))

In [18]:
start = root[:1]

In [19]:
end = root[-1:]

In [20]:
print(start[0].tag)

child0


In [21]:
print(end[0].tag)

child3


在 ElementTree 1.3 和 lxml 2.0 之前，您还可以检查 Element 的真值以查看它是否有子元素，即子元素列表是否为空

In [22]:
if root:
    print("The root element has children.")

The root element has children.


C:\Users\98782\AppData\Local\Temp\ipykernel_14000\1721921444.py:1: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if root:


这不再受支持，因为人们倾向于期望“某物”评估为 True 并期望 Elements 是“某物”，无论他们是否有孩子。因此，许多用户感到惊讶的是，在上面这样的 if 语句中任何元素都会计算为 False。相反，使用 len(element)，它更明确且不易出错

In [23]:
print(etree.iselement(root))

True


In [24]:
if len(root):
    print("The root element has children")

The root element has children


还有另一个重要的情况，lxml 中的 Elements 的行为（2.0 及更高版本）与列表和原始 ElementTree（版本 1.3 或 Python 2.7/3.2 之前）的行为不同

In [25]:
for child in root:
    print(child.tag)

child0
child1
child2
child3


In [26]:
root[0] = root[-1]  # this moves the element in lxml.etree!

In [27]:
for child in root:
    print(child.tag)

child3
child1
child2


在此示例中，最后一个元素被移动到不同的位置，而不是被复制，即，当它被放置在不同的位置时，它会自动从之前的位置删除。在列表中，对象可以同时出现在多个位置，上面的赋值只是将项目引用复制到第一个位置，以便两者都包含完全相同的项目

In [28]:
l = [0, 1, 2, 3]

In [29]:
l[0] = l[-1]

In [30]:
l

[3, 1, 2, 3]

请注意，在原始 ElementTree 中，单个 Element 对象可以位于任意数量的树中的任意位置，这允许与列表进行相同的复制操作。明显的缺点是，对此类元素的修改将应用于它出现在树中的所有位置，这可能是有意的，也可能不是有意的

这种差异的好处是 lxml.etree 中的元素始终只有一个父级，可以通过 getparent() 方法查询。原始 ElementTree 不支持这一点。

In [31]:
root is root[0].getparent()

True

如果要将元素复制到 lxml.etree 中的不同位置，请考虑使用 Python 标准库中的复制模块创建独立的深层复制：

In [32]:
from copy import deepcopy

In [33]:
element = etree.Element("neu")

In [34]:
element.append(deepcopy(root[1]))

In [35]:
print(element[0].tag)

child1


In [36]:
print([c.tag for c in root])

['child3', 'child1', 'child2']


元素的兄弟（或邻居）作为下一个和上一个元素进行访问

In [37]:
root[0] is root[1].getprevious()

True

In [38]:
root[1] is root[0].getnext()

True

元素以字典形式携带属性

XML 元素支持属性。您可以直接在元素工厂中创建它们

In [39]:
root = etree.Element("root", interesting="totally")

In [40]:
etree.tostring(root)

b'<root interesting="totally"/>'

In [41]:
prettyprint(root)

<root interesting="totally"/>


属性只是无序的名称-值对，因此处理它们的一种非常方便的方法是通过 Elements 的类似字典的接口

In [42]:
print(root.get("interesting"))

totally


In [43]:
print(root.get("hello"))

None


In [44]:
root.set("hello", "Huhu")

In [45]:
root.get("hello")

'Huhu'

In [46]:
prettyprint(root)

<root interesting="totally" hello="Huhu"/>


In [47]:
sorted(root.keys())

['hello', 'interesting']

In [48]:
root.items()

[('interesting', 'totally'), ('hello', 'Huhu')]

In [49]:
for name, value in sorted(root.items()):
    print("%s = %s" % (name, value))

hello = Huhu
interesting = totally


对于您想要进行项目查找或有其他原因获取“真实”类似字典的对象的情况，例如为了传递它，您可以使用 attrib 属性

In [50]:
attributes = root.attrib

In [51]:
attributes["interesting"]

'totally'

In [52]:
print(attributes.get("no-such-attribute"))

None


In [53]:
attributes['hello'] = "Guten Tag"

In [54]:
attributes['hello']

'Guten Tag'

In [55]:
root.get('hello')

'Guten Tag'

请注意，attrib 是一个由元素本身支持的类似字典的对象。这意味着对元素的任何更改都会反映在 attrib 中，反之亦然。它还意味着只要 XML 树的元素之一的属性正在使用，它就会在内存中保持活动状态。要获取不依赖于 XML 树的属性的独立快照，请将其复制到字典中：

In [56]:
d = dict(root.attrib)
d

{'interesting': 'totally', 'hello': 'Guten Tag'}

In [57]:
sorted(d.items())

[('hello', 'Guten Tag'), ('interesting', 'totally')]

元素包含文本

In [58]:
root = etree.Element("root")

In [59]:
root.text = "TEXT"

In [60]:
print(root.text)

TEXT


In [61]:
prettyprint(root)

<root>TEXT</root>


在许多 XML 文档（以数据为中心的文档）中，这是唯一可以找到文本的地方。它由树层次结构最底部的叶标签封装。

但是，如果 XML 用于标记文本文档（例如 (X)HTML），则文本也可以出现在不同元素之间，就在树的中间

``` html
<html><body>Hello<br/>World</body></html>
```

此处，`<br/> `标记被文本包围。这通常称为文档样式或混合内容 XML。元素通过其尾部属性支持这一点。它包含直接跟随该元素的文本，直到 XML 树中的下一个元素

In [62]:
html = etree.Element("html")

In [63]:
body = etree.SubElement(html, "body")

In [64]:
body.text = "TEXT"

In [65]:
prettyprint(html)

<html>
  <body>TEXT</body>
</html>


In [66]:
br = etree.SubElement(body, "br")

In [67]:
prettyprint(html)

<html>
  <body>TEXT<br/></body>
</html>


In [68]:
br.tail = "TALL"

In [69]:
prettyprint(html)

<html>
  <body>TEXT<br/>TALL</body>
</html>


.text 和 .tail 这两个属性足以表示 XML 文档中的任何文本内容。这样，除了 Element 类之外，ElementTree API 不需要任何特殊的文本节点，这些节点往往会造成干扰（正如您可能从经典 DOM API 中知道的那样）

然而，在某些情况下，尾部文本也会产生妨碍。例如，当您从树中序列化一个元素时，您并不总是希望结果中包含其尾部文本（尽管您仍然希望其子元素的尾部文本）。为此，tostring() 函数接受关键字参数 with_tail

In [70]:
etree.tostring(br)

b'<br/>TALL'

In [71]:
etree.tostring(br, with_tail=False)

b'<br/>'

如果您只想读取文本，即没有任何中间标签，则必须以正确的顺序递归连接所有文本和尾部属性。 tostring() 函数再次发挥作用，这次使用了 method 关键字

In [72]:
etree.tostring(html, method="text")

b'TEXTTALL'

使用 XPath 查找文本

提取树的文本内容的另一种方法是 XPath，它还允许您将单独的文本块提取到列表中

In [73]:
html.xpath("string()")

'TEXTTALL'

In [74]:
html.xpath("//text()")

['TEXT', 'TALL']

如果你想更频繁地使用它，你可以将它包装在一个函数中

In [75]:
build_text_list = etree.XPath("//text()")

In [76]:
print(build_text_list(html))

['TEXT', 'TALL']


请注意，XPath 返回的字符串结果是一个特殊的“智能”对象，它知道其来源。你可以通过 getparent() 方法询问它来自哪里，就像使用 Elements 一样

In [77]:
texts = build_text_list(html)

In [78]:
print(texts[0])

TEXT


In [79]:
parent = texts[0].getparent()

In [80]:
print(parent.tag)

body


In [81]:
print(texts[1])

TALL


In [82]:
print(texts[1].getparent().tag)

br


您还可以确定它是普通文本内容还是尾部文本

In [83]:
print(texts[0].is_text)

True


In [84]:
print(texts[1].is_text)

False


In [85]:
print(texts[1].is_tail)

True


虽然这适用于 text() 函数的结果，但 lxml 不会告诉您由 XPath 函数 string() 或 concat() 构造的字符串值的来源

In [86]:
stringify = etree.XPath("string()")

In [87]:
print(stringify(html))

TEXTTALL


树迭代

对于上面这样的问题，如果您想要递归地遍历树并对其元素执行某些操作，树迭代是一个非常方便的解决方案。 Elements 为此目的提供了一个树迭代器。它按文档顺序生成元素，即如果将树序列化为 XML，则按其标签出现的顺序：

In [88]:
root = etree.Element("root")

In [89]:
etree.SubElement(root, "child").text = "Child 1"

In [90]:
etree.SubElement(root, "child").text = "Child 2"

In [91]:
etree.SubElement(root, "another").text = "Child 3"

In [92]:
prettyprint(root)

<root>
  <child>Child 1</child>
  <child>Child 2</child>
  <another>Child 3</another>
</root>


In [93]:
for element in root.iter():
    print(f"{element.tag} - {element.text}")

root - None
child - Child 1
child - Child 2
another - Child 3


如果您知道您只对单个标签感兴趣，则可以将其名称传递给 iter() 以让它为您过滤。从 lxml 3.0 开始，您还可以传递多个标签以在迭代期间拦截多个标签。

In [94]:
for  element in root.iter("child"):
    print(f"{element.tag} - {element.text}")

child - Child 1
child - Child 2


In [95]:
for element in root.iter("another", "child"):
    print(f"{element.tag} - {element.text}")

child - Child 1
child - Child 2
another - Child 3


默认情况下，迭代会生成树中的所有节点，包括处理指令、注释和实体实例。如果你想确保只返回 Element 对象，你可以将 Element 工厂作为标签参数传递

In [96]:
root.append(etree.Entity("#234"))

In [97]:
root.append(etree.Comment("some comment"))

In [98]:
prettyprint(root)

<root><child>Child 1</child><child>Child 2</child><another>Child 3</another>&#234;<!--some comment--></root>


In [99]:
for element in root.iter():
    if  isinstance(element.tag, str):
        print(f"{element.tag} - {element.text}")
    else:
        print(f"SPECIAL: {element} - {element.text}")

root - None
child - Child 1
child - Child 2
another - Child 3
SPECIAL: &#234; - &#234;
SPECIAL: <!--some comment--> - some comment


In [100]:
for element in root.iter(tag=etree.Element):
    print(f"{element.tag} - {element.text}")

root - None
child - Child 1
child - Child 2
another - Child 3


In [101]:
for element in root.iter(tag=etree.Entity):
    print(element.text)

&#234;


请注意，传递通配符“*”标记名称还将生成所有 Element 节点（并且仅生成元素）。 在 lxml.etree 中，元素为树中的所有方向提供了进一步的迭代器：孩子、父母（或更确切地说祖先）和兄弟姐妹。

序列化

序列化通常使用返回字符串的 tostring() 函数，或写入文件、类文件对象或 URL（通过 FTP PUT 或 HTTP POST）的 ElementTree.write() 方法。两个调用都接受相同的关键字参数，例如用于格式化输出的 Pretty_print 或编码以选择除纯 ASCII 之外的特定输出编码

In [102]:
root = etree.XML('<root><a><b/></a></root>')

In [103]:
prettyprint(root)

<root>
  <a>
    <b/>
  </a>
</root>


In [104]:
etree.tostring(root)

b'<root><a><b/></a></root>'

In [105]:
xml_string = etree.tostring(root, xml_declaration=True)

In [106]:
print(xml_string)

b"<?xml version='1.0' encoding='ASCII'?>\n<root><a><b/></a></root>"


In [107]:
latin1_bytesstring = etree.tostring(root, encoding="iso8859-1")

In [108]:
print(latin1_bytesstring.decode('iso8859-1'),end='')

<?xml version='1.0' encoding='iso8859-1'?>
<root><a><b/></a></root>

In [109]:
print(etree.tostring(root, pretty_print=True).decode(), end="")

<root>
  <a>
    <b/>
  </a>
</root>


请注意，漂亮的打印会在末尾添加换行符。因此，我们在这里使用 end='' 选项来防止 print() 函数添加另一个换行符

为了对漂亮打印进行更细粒度的控制，您可以在序列化树之前使用 indent() 函数（在 lxml 4.5 中添加）向树添加空格缩进

In [110]:
root = etree.XML('<root><a><b/>\n</a></root>')
print(etree.tostring(root).decode())

<root><a><b/>
</a></root>


In [111]:
etree.indent(root)

In [112]:
print(etree.tostring(root).decode())

<root>
  <a>
    <b/>
  </a>
</root>


In [113]:
root.text

'\n  '

In [114]:
root[0].text

'\n    '

In [115]:
etree.indent(root, space="    ")

In [116]:
print(etree.tostring(root).decode())

<root>
    <a>
        <b/>
    </a>
</root>


In [117]:
etree.indent(root, space="\t")

In [118]:
etree.tostring(root)

b'<root>\n\t<a>\n\t\t<b/>\n\t</a>\n</root>'

In [119]:
prettyprint(root)

<root>
	<a>
		<b/>
	</a>
</root>


在 lxml 2.0 及更高版本以及 xml.etree 中，序列化函数可以执行的不仅仅是 XML 序列化。您可以通过传递method关键字序列化为HTML或提取文本内容

In [120]:
root = etree.XML('<html><head/><body><p>Hello<br/>World</p></body></html>')

In [121]:
etree.tostring(root)  # default: method = 'xml'

b'<html><head/><body><p>Hello<br/>World</p></body></html>'

In [122]:
etree.tostring(root, method='xml')  # same as above

b'<html><head/><body><p>Hello<br/>World</p></body></html>'

In [123]:
etree.tostring(root, method='html')

b'<html><head></head><body><p>Hello<br>World</p></body></html>'

In [124]:
prettyprint(root, method="html")

<html>
<head></head>
<body><p>Hello<br>World</p></body>
</html>


In [125]:
etree.tostring(root, method='text')

b'HelloWorld'

对于 XML 序列化，纯文本序列化的默认编码是 ASCII

In [126]:
br = next(root.iter('br'))  # get first result of iteration

In [127]:
br.tail = 'World'

In [128]:
etree.tostring(root, method='text')  # doctest: + ELLIPSIS

b'HelloWorld'

In [129]:
etree.tostring(root, method='text', encoding="UTF-8")

b'HelloWorld'

在这里，序列化为 Python 文本字符串而不是字节字符串可能会变得很方便。只需传递名称“unicode”作为编码：

In [130]:
etree.tostring(root, encoding='unicode', method='text')

'HelloWorld'

In [131]:
etree.tostring(root, encoding='unicode')

'<html><head/><body><p>Hello<br/>World</p></body></html>'

ElementTree 类

ElementTree 主要是一个带有根节点的树的文档包装器。它提供了几种用于序列化和一般文档处理的方法。

从字符串和文件中解析

lxml.etree 支持以多种方式解析来自所有重要源的 XML，即字符串、文件、URL (http/ftp) 和类似文件的对象。主要的解析函数是 fromstring() 和 parse()，两者都以源作为第一个参数进行调用。默认情况下，它们使用标准解析器，但您始终可以传递不同的解析器作为第二个参数

fromstring() 函数

fromstring() 函数是解析字符串的最简单方法

In [132]:
some_xml_data = "<root>data</root>"

In [133]:
root = etree.fromstring(some_xml_data)

In [134]:
print(root.tag)

root


In [135]:
etree.tostring(root)

b'<root>data</root>'

In [136]:
prettyprint(root)

<root>data</root>


XML() 函数

XML() 函数的行为类似于 fromstring() 函数，但通常用于将 XML 文字直接写入源中

In [137]:
root = etree.XML("<root>data</root>")

In [138]:
root.tag

'root'

In [139]:
etree.tostring(root)

b'<root>data</root>'

对于 HTML 文字还有一个相应的函数 HTML()。

In [140]:
root = etree.HTML("<p>data</p>")

In [141]:
etree.tostring(root)

b'<html><body><p>data</p></body></html>'

parse() 函数

arse() 函数用于解析文件和类文件对象。 

作为此类文件对象的示例，以下代码使用 BytesIO 类从字符串而不是外部文件中读取。然而，在现实生活中，您显然会避免这样做并使用上面的 fromstring() 等字符串解析函数。

In [142]:
from io import BytesIO

In [143]:
some_file_or_file_like_object = BytesIO(b"<root>data</root>")

In [144]:
tree = etree.parse(some_file_or_file_like_object)

In [145]:
etree.tostring(tree)

b'<root>data</root>'

请注意，parse() 返回一个 ElementTree 对象，而不是字符串解析器函数那样的 Element 对象

In [146]:
root = tree.getroot()

In [147]:
root.tag

'root'

In [148]:
etree.tostring(root)

b'<root>data</root>'

解析器对象

默认情况下，lxml.etree 使用具有默认设置的标准解析器。如果要配置解析器，可以创建一个新实例：

In [149]:
parser = etree.XMLParser(remove_blank_text=True)

这将创建一个解析器，在解析时删除标记之间的空文本，如果您知道纯空白内容对数据没有意义，则可以减小树的大小并避免悬空尾部文本。一个例子

In [150]:
 root = etree.XML("<root>  <a/>   <b>  </b>     </root>", parser)

请注意，`<b>` 标记内的空白内容并未被删除，因为叶元素的内容往往是数据内容（即使是空白）。您可以通过遍历树在额外的步骤中轻松删除它

In [151]:
etree.tostring(root)

b'<root><a/><b>  </b></root>'